In [10]:
import pandas as pd

In [11]:
neighbourhoods = pd.read_csv("D:/DataSets/Neighbourhoods/Neighbourhoods.csv")

In [12]:
neighbourhoods["googleNeighbourhoodGeoLocation"] = None

In [13]:
neighbourhoods

,neighbourhoodID,neighbourhoodGeoLocation,googleNeighbourhoodName,googleNeighbourhoodGeoLocation
0,0,POINT(39.36253019999999 -76.6595727),MOUNT WASHINGTON,None
1,1,POINT(39.2842097 -76.593611),FELLS POINT,None
2,2,POINT(39.3226705 -76.5846633),BELAIR - EDISON,None
3,3,POINT(39.292007 -76.6167212),DOWNTOWN,None
4,4,POINT(39.3481652 -76.6831397),ARLINGTON,None
...,...,...,...,...
76,76,POINT(39.30592439999999 -76.6650946),NW COMMUNITY ACTION,None
77,77,POINT(39.3041275 -76.6441591),DRUID HILL PARK,None
78,78,POINT(39.3625548 -76.544488),WOODRING,None
79,79,POINT(39.2208578 -76.5863882),CURTIS BAY,None


In [15]:
import googlemaps
import re

def retrieve_neighbourhood(property_latitude, property_longitude):
    API_KEY = open('C:/Users/seanl/googlemaps_api.txt', 'r').read()
    gmaps = googlemaps.Client(API_KEY)

    # Perform reverse geocoding to get location details
    if property_latitude and property_longitude:
        reverse_geocode_result = gmaps.reverse_geocode((property_latitude, property_longitude))

        # Extract neighborhood information and coordinates if available
        neighborhood_info = None

        if reverse_geocode_result:
            for result in reverse_geocode_result:
                for component in result['address_components']:
                    if 'neighborhood' in component['types']:
                        neighborhood = component['long_name']
                        neighborhood_coords = result['geometry']['location']
                        neighborhood_info = {
                            'name': neighborhood,
                            'coordinates': neighborhood_coords
                        }
    
            if neighborhood_info:
                print(f'Neighborhood: {neighborhood_info["name"]}, Coordinates: {neighborhood_info["coordinates"]}')
                neighborhood_coords = str(neighborhood_coords)
                numbers = re.findall(r"[-+]?\d*\.\d+|\d+", neighborhood_coords)

                # Join the numbers with a space
                formatted_coords = ' '.join(numbers)
                location = f"POINT({formatted_coords})"
                return neighborhood, location
            else:
                print('Neighborhood information not found.')
                return None
    else:
        print('Invalid coordinates provided.')
        return None

In [16]:
import time
import re

for i in range((len(neighbourhoods))):
    match = re.search(r'POINT\(([-\d.]+) ([-\d.]+)\)', neighbourhoods["neighbourhoodGeoLocation"][i])

    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
    
    response = retrieve_neighbourhood(latitude,longitude)
    neighbourhoods.loc[i, 'googleNeighbourhoodName'] = response[0]
    neighbourhoods.loc[i, 'googleNeighbourhoodGeoLocation'] = response[1]
    time.sleep(2)

Neighborhood: Mount Washington, Coordinates: {'lat': 39.370859, 'lng': -76.6701475}
Neighborhood: Fells Point, Coordinates: {'lat': 39.2842097, 'lng': -76.593611}
Neighborhood: Belair - Edison, Coordinates: {'lat': 39.3216614, 'lng': -76.5737173}
Neighborhood: Downtown, Coordinates: {'lat': 39.292007, 'lng': -76.6167212}
Neighborhood: Arlington, Coordinates: {'lat': 39.3416271, 'lng': -76.68169619999999}
Neighborhood: Carroll, Coordinates: {'lat': 39.2817753, 'lng': -76.69324379999999}
Neighborhood: Purnell, Coordinates: {'lat': 39.320533, 'lng': -76.7065867}
Neighborhood: Original Northwood, Coordinates: {'lat': 39.3406246, 'lng': -76.59649999999999}
Neighborhood: Original Northwood, Coordinates: {'lat': 39.3406246, 'lng': -76.59649999999999}
Neighborhood: Locust Point, Coordinates: {'lat': 39.2680512, 'lng': -76.59105919999999}
Neighborhood: Lakeland, Coordinates: {'lat': 39.2530783, 'lng': -76.64659019999999}
Neighborhood: Druid Hill Park, Coordinates: {'lat': 39.3056685, 'lng': -76

In [24]:
neighbourhoods.to_csv("D:/DataSets/Neighbourhoods/NeighbourhoodsV2.csv", index = False)

In [19]:
addresses = pd.read_csv("D:/DataSets/backup_finished/addresses.csv")

In [20]:
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

# Use the map function to fill in the empty googleNeighbourhoodGeoLocation in addresses
addresses['googleNeighbourhoodGeoLocation'] = addresses['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

In [22]:
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

# Use the map function to fill in the empty googleNeighbourhoodGeoLocation in addresses
addresses['googleNeighbourhoodName'] = addresses['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

In [48]:
addresses = addresses.drop("neighbourhoodGeoLocation", axis = 1)
addresses = addresses.drop("googleNeighbourhoodName", axis = 1)

In [49]:
addresses.to_csv("D:/DataSets/DatabaseTables/addresses.csv", index = False)

In [164]:
data_no_duplicates = neighbourhoods.drop_duplicates('googleNeighbourhoodName', keep='first')
data_no_duplicates = data_no_duplicates.reset_index(drop = True)

In [165]:
data_no_duplicates = data_no_duplicates.drop("neighbourhoodGeoLocation", axis = 1)
data_no_duplicates = data_no_duplicates.drop("googleNeighbourhoodName", axis = 1)

In [166]:
data_no_duplicates = data_no_duplicates.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [171]:
data_no_duplicates['neighbourhoodID'] = range(len(data_no_duplicates))

In [172]:
data_no_duplicates

,neighbourhoodID,neighbourhoodGeoLocation
0,0,POINT(39.370859 -76.6701475)
1,1,POINT(39.2842097 -76.593611)
2,2,POINT(39.3216614 -76.5737173)
3,3,POINT(39.292007 -76.6167212)
4,4,POINT(39.3416271 -76.68169619999999)
5,5,POINT(39.2817753 -76.69324379999999)
6,6,POINT(39.320533 -76.7065867)
7,7,POINT(39.3406246 -76.59649999999999)
8,8,POINT(39.2680512 -76.59105919999999)
9,9,POINT(39.2530783 -76.64659019999999)


In [173]:
data_no_duplicates.to_csv("D:/DataSets/DatabaseTables/NeighbourhoodsV4.csv", index = False)

# Remap for each of the services

# Banks

In [154]:
# banks
banks = pd.read_csv("D:/DataSets/NewServices//banks.csv")

In [155]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

banks['googleNeighbourhoodGeoLocation'] = banks['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

banks['googleNeighbourhoodName'] = banks['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

banks.to_csv("D:/DataSets/ServicesFullInformation/banksV2.csv", index = False)

In [156]:
banks = banks.drop("neighbourhoodGeoLocation", axis = 1)
banks = banks.drop("googleNeighbourhoodName", axis = 1)

In [157]:
banks = banks.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [158]:
banks.to_csv("D:/DataSets/DatabaseTables/banks.csv", index = False)

# Bars

In [55]:
bars = pd.read_csv("D:/DataSets/NewServices//bars.csv")

In [56]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

bars['googleNeighbourhoodGeoLocation'] = bars['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

bars['googleNeighbourhoodName'] = bars['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

bars.to_csv("D:/DataSets/ServicesFullInformation/barsV2.csv", index = False)

In [58]:
bars = bars.drop("neighbourhoodGeoLocation", axis = 1)
bars = bars.drop("googleNeighbourhoodName", axis = 1)

In [111]:
bars = bars.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [113]:
bars.to_csv("D:/DataSets/DatabaseTables/bars.csv", index = False)

# Beauty

In [61]:
# banks
beauty_salons = pd.read_csv("D:/DataSets/NewServices/beauty_salons.csv")

In [62]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

beauty_salons['googleNeighbourhoodGeoLocation'] = beauty_salons['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

beauty_salons['googleNeighbourhoodName'] = beauty_salons['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

beauty_salons.to_csv("D:/DataSets/ServicesFullInformation/beauty_salonsV2.csv", index = False)

In [63]:
beauty_salons = beauty_salons.drop("neighbourhoodGeoLocation", axis = 1)
beauty_salons = beauty_salons.drop("googleNeighbourhoodName", axis = 1)

In [109]:
beauty_salons = beauty_salons.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [110]:
beauty_salons.to_csv("D:/DataSets/DatabaseTables/beauty_salons.csv", index = False)

# Bus Stations

In [65]:
# banks
bus_stations = pd.read_csv("D:/DataSets/NewServices//bus_stations.csv")

In [66]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

bus_stations['googleNeighbourhoodGeoLocation'] = bus_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

bus_stations['googleNeighbourhoodName'] = bus_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

bus_stations.to_csv("D:/DataSets/ServicesFullInformation/bus_stationsV2.csv", index = False)

In [67]:
bus_stations = bus_stations.drop("neighbourhoodGeoLocation", axis = 1)
bus_stations = bus_stations.drop("googleNeighbourhoodName", axis = 1)

In [107]:
bus_stations = bus_stations.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [108]:
bus_stations.to_csv("D:/DataSets/DatabaseTables/bus_stations.csv", index = False)

# Cafes

In [69]:
# banks
cafes = pd.read_csv("D:/DataSets/NewServices/cafes.csv")

In [70]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

cafes['googleNeighbourhoodGeoLocation'] = cafes['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

cafes['googleNeighbourhoodName'] = cafes['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

cafes.to_csv("D:/DataSets/ServicesFullInformation/cafes.csv", index = False)

In [71]:
cafes = cafes.drop("neighbourhoodGeoLocation", axis = 1)
cafes = cafes.drop("googleNeighbourhoodName", axis = 1)

In [105]:
cafes = cafes.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [106]:
cafes.to_csv("D:/DataSets/DatabaseTables/cafes.csv", index = False)

# Fire Stations

In [73]:
# banks
fire_stations = pd.read_csv("D:/DataSets/NewServices//fire_stations.csv")

In [74]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

fire_stations['googleNeighbourhoodGeoLocation'] = fire_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

fire_stations['googleNeighbourhoodName'] = fire_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

fire_stations.to_csv("D:/DataSets/ServicesFullInformation/banksV2.csv", index = False)

In [75]:
fire_stations = fire_stations.drop("neighbourhoodGeoLocation", axis = 1)
fire_stations = fire_stations.drop("googleNeighbourhoodName", axis = 1)

In [103]:
fire_stations = fire_stations.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [104]:
fire_stations.to_csv("D:/DataSets/DatabaseTables/fire_stations.csv", index = False)

# Gyms

In [77]:
# banks
gyms = pd.read_csv("D:/DataSets/NewServices//gyms.csv")

In [78]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

gyms['googleNeighbourhoodGeoLocation'] = gyms['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

gyms['googleNeighbourhoodName'] = gyms['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

gyms.to_csv("D:/DataSets/ServicesFullInformation/gymsV2.csv", index = False)

In [79]:
gyms = gyms.drop("neighbourhoodGeoLocation", axis = 1)
gyms = gyms.drop("googleNeighbourhoodName", axis = 1)

In [101]:
gyms = gyms.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [102]:
gyms.to_csv("D:/DataSets/DatabaseTables/gyms.csv", index = False)

# Hospitals

In [81]:
# banks
hospitals = pd.read_csv("D:/DataSets/NewServices//hospitals.csv")

In [82]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

hospitals['googleNeighbourhoodGeoLocation'] = hospitals['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

hospitals['googleNeighbourhoodName'] = hospitals['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

hospitals.to_csv("D:/DataSets/ServicesFullInformation/hospitalsV2.csv", index = False)

In [83]:
hospitals = hospitals.drop("neighbourhoodGeoLocation", axis = 1)
hospitals = hospitals.drop("googleNeighbourhoodName", axis = 1)

In [98]:
hospitals = hospitals.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [100]:
hospitals.to_csv("D:/DataSets/DatabaseTables/hospitals.csv", index = False)

# Night Clubs

In [85]:
# banks
night_clubs = pd.read_csv("D:/DataSets/NewServices//night_clubs.csv")

In [86]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

night_clubs['googleNeighbourhoodGeoLocation'] = night_clubs['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

night_clubs['googleNeighbourhoodName'] = night_clubs['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

night_clubs.to_csv("D:/DataSets/ServicesFullInformation/night_clubsV2.csv", index = False)

In [87]:
night_clubs = night_clubs.drop("neighbourhoodGeoLocation", axis = 1)
night_clubs = night_clubs.drop("googleNeighbourhoodName", axis = 1)

In [95]:
night_clubs = night_clubs.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [97]:
night_clubs.to_csv("D:/DataSets/DatabaseTables/night_clubs.csv", index = False)

# Parks

In [116]:
# banks
parks = pd.read_csv("D:/DataSets/NewServices//parks.csv")

In [117]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

parks['googleNeighbourhoodGeoLocation'] = parks['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

parks['googleNeighbourhoodName'] = parks['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

parks.to_csv("D:/DataSets/ServicesFullInformation/parksV2.csv", index = False)

In [118]:
parks = parks.drop("neighbourhoodGeoLocation", axis = 1)
parks = parks.drop("googleNeighbourhoodName", axis = 1)

In [119]:
parks = parks.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [120]:
parks.to_csv("D:/DataSets/DatabaseTables/parks.csv", index = False)

# Pharmacies

In [121]:
# banks
pharmacies = pd.read_csv("D:/DataSets/NewServices/pharmacies.csv")

In [122]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

pharmacies['googleNeighbourhoodGeoLocation'] = pharmacies['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

pharmacies['googleNeighbourhoodName'] = pharmacies['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

pharmacies.to_csv("D:/DataSets/ServicesFullInformation/pharmaciesV2.csv", index = False)

In [123]:
pharmacies = pharmacies.drop("neighbourhoodGeoLocation", axis = 1)
pharmacies = pharmacies.drop("googleNeighbourhoodName", axis = 1)

In [124]:
pharmacies = pharmacies.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [125]:
pharmacies.to_csv("D:/DataSets/DatabaseTables/pharmacies.csv", index = False)

# Police Stations

In [126]:
# banks
police_stations = pd.read_csv("D:/DataSets/NewServices/police_stations.csv")

In [127]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

police_stations['googleNeighbourhoodGeoLocation'] = police_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

police_stations['googleNeighbourhoodName'] = police_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

police_stations.to_csv("D:/DataSets/ServicesFullInformation/police_stationsV2.csv", index = False)

In [128]:
police_stations = police_stations.drop("neighbourhoodGeoLocation", axis = 1)
police_stations = police_stations.drop("googleNeighbourhoodName", axis = 1)

In [129]:
police_stations = police_stations.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [130]:
police_stations.to_csv("D:/DataSets/DatabaseTables/police_stations.csv", index = False)

# Restaurants

In [131]:
# banks
restaurants = pd.read_csv("D:/DataSets/NewServices/restaurants.csv")

In [132]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

restaurants['googleNeighbourhoodGeoLocation'] = restaurants['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

restaurants['googleNeighbourhoodName'] = restaurants['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

restaurants.to_csv("D:/DataSets/ServicesFullInformation/restaurantsV2.csv", index = False)

In [133]:
restaurants = restaurants.drop("neighbourhoodGeoLocation", axis = 1)
restaurants = restaurants.drop("googleNeighbourhoodName", axis = 1)

In [134]:
restaurants = restaurants.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [135]:
restaurants.to_csv("D:/DataSets/DatabaseTables/restaurants.csv", index = False)

# Supermarkets

In [136]:
# banks
supermarkets = pd.read_csv("D:/DataSets/NewServices/supermarkets.csv")

In [137]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

supermarkets['googleNeighbourhoodGeoLocation'] = supermarkets['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

supermarkets['googleNeighbourhoodName'] = supermarkets['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

supermarkets.to_csv("D:/DataSets/ServicesFullInformation/supermarketsV2.csv", index = False)

In [138]:
supermarkets = supermarkets.drop("neighbourhoodGeoLocation", axis = 1)
supermarkets = supermarkets.drop("googleNeighbourhoodName", axis = 1)

In [139]:
supermarkets = supermarkets.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [140]:
supermarkets.to_csv("D:/DataSets/DatabaseTables/supermarkets.csv", index = False)

# Train Stations

In [149]:
# banks
train_stations = pd.read_csv("D:/DataSets/NewServices/train_stations.csv")

In [150]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

train_stations['googleNeighbourhoodGeoLocation'] = train_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

train_stations['googleNeighbourhoodName'] = train_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

train_stations.to_csv("D:/DataSets/ServicesFullInformation/train_stationsV2.csv", index = False)

In [151]:
train_stations = train_stations.drop("neighbourhoodGeoLocation", axis = 1)
train_stations = train_stations.drop("googleNeighbourhoodName", axis = 1)

In [152]:
train_stations = train_stations.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [153]:
train_stations.to_csv("D:/DataSets/DatabaseTables/train_stations.csv", index = False)

# Transit Stations

In [159]:
# banks
transit_stations = pd.read_csv("D:/DataSets/NewServices//transit_stations.csv")

In [160]:
# create neighbourhood mapping
neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodGeoLocation'].to_dict()

transit_stations['googleNeighbourhoodGeoLocation'] = transit_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

neighbourhood_mapping = neighbourhoods.set_index('neighbourhoodGeoLocation')['googleNeighbourhoodName'].to_dict()

transit_stations['googleNeighbourhoodName'] = transit_stations['neighbourhoodGeoLocation'].map(neighbourhood_mapping)

transit_stations.to_csv("D:/DataSets/ServicesFullInformation/transit_stationsV2.csv", index = False)

In [161]:
transit_stations = transit_stations.drop("neighbourhoodGeoLocation", axis = 1)
transit_stations = transit_stations.drop("googleNeighbourhoodName", axis = 1)

In [162]:
transit_stations = transit_stations.rename(columns = {"googleNeighbourhoodGeoLocation": "neighbourhoodGeoLocation"})

In [163]:
transit_stations.to_csv("D:/DataSets/DatabaseTables/transit_stations.csv", index = False)